Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploying a web service to Azure Kubernetes Service (AKS)
In this notebook, we show the following steps for deploying a web service using AML:
- Create an image
- Test image locally
- Provision an AKS cluster (one time action)
- Deploy the service
- Test the web service
- Scale up the service

In [ ]:
import os
import pandas as pd
from utilities import text_to_json
import requests
import numpy as np
import json
import subprocess
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.core.workspace import Workspace
from azureml.core.conda_dependencies import CondaDependencies
from dotenv import set_key, get_key, find_dotenv
from utilities import _get_auth

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)

AML will use the following information to create an image, provision a cluster and deploy a service. Replace the values in the following cell with your information.

In [ ]:
image_name = "YOUR_IMAGE_NAME"
aks_service_name = "YOUR_AKS_SERVICE_NAME"
aks_name = "YOUR_AKS_NAME"
aks_location = "YOUR_AKS_LOCATION"

In [ ]:
set_key(env_path, "image_name", image_name)
set_key(env_path, "aks_service_name", aks_service_name)
set_key(env_path, "aks_name", aks_name)
set_key(env_path, "aks_location", aks_location)

## Get workspace
Load existing workspace from the config file.

In [ ]:
ws = Workspace.from_config(auth=_get_auth(env_path))
print(ws.name, ws.resource_group, ws.location, sep="\n")

## Load model

In [ ]:
model_name = 'question_match_model'
model_version = int(get_key(env_path, 'model_version'))
model = Model(ws, name=model_name, version=model_version)
print(model.name, model.version)

## Create an image
We will now modify the `score.py` created in the previous notebook for the `init()` function to use the model we registered to the workspace earlier.

In [ ]:
%%writefile score.py

import sys
import pandas as pd
import json
from duplicate_model import DuplicateModel
import logging
import timeit as t
from azureml.core.model import Model
sys.path.append('./scripts/')

def init():
    logger = logging.getLogger("scoring_script")
    global model
    model_name = 'question_match_model'
    model_path = Model.get_model_path(model_name)
    questions_path = './data_folder/questions.tsv'
    start = t.default_timer()
    model = DuplicateModel(model_path, questions_path)
    end = t.default_timer()
    loadTimeMsg = "Model loading time: {0} ms".format(round((end-start)*1000, 2))
    logger.info(loadTimeMsg)

def run(body):
    logger = logging.getLogger("scoring_script")
    json_load_text = json.loads(body)
    text_to_score = json_load_text['input']
    start = t.default_timer()
    resp = model.score(text_to_score) 
    end = t.default_timer()
    logger.info("Prediction took {0} ms".format(round((end-start)*1000, 2)))
    return(json.dumps(resp))

Let's specifiy the conda and pip dependencies for the image.

In [ ]:
conda_pack = ["scikit-learn==0.19.1", "pandas==0.23.3"]
requirements = ["lightgbm==2.1.2", "azureml-defaults==1.0.10"]

In [ ]:
lgbmenv = CondaDependencies.create(conda_packages=conda_pack, pip_packages=requirements)

with open("lgbmenv.yml", "w") as f:
    f.write(lgbmenv.serialize_to_string())

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script="score.py",
    runtime="python",
    conda_file="lgbmenv.yml",
    description="Image with lightgbm model",
    tags={"area": "text", "type": "lightgbm"},
    dependencies=[
        "./data_folder/questions.tsv",
        "./duplicate_model.py",
        "./scripts/ItemSelector.py",
    ],
)

image = ContainerImage.create(
    name=image_name,
    # this is the model object
    models=[model],
    image_config=image_config,
    workspace=ws,
)

In [ ]:
%%time
image.wait_for_creation(show_output = True)

In [ ]:
print(image.name, image.version)

In [ ]:
image_version = str(image.version)
set_key(env_path, "image_version", image_version)

You can find the logs of image creation in the following location.

In [ ]:
image.image_build_log_uri

## Test image locally

Now, let's use one of the duplicate questions to test our image.

In [ ]:
dupes_test_path = './data_folder/dupes_test.tsv'
dupes_test = pd.read_csv(dupes_test_path, sep='\t', encoding='latin1')
text_to_score = dupes_test.iloc[0,4]
text_to_score

In [ ]:
jsontext = text_to_json(text_to_score)

In [ ]:
%%time
image.run(input_data=jsontext)

## Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it. Let's first check if there are enough cores in the subscription for the cluster .

In [ ]:
vm_dict = {
    "Dv2": {
        "size": "Standard_D4_v2",
        "cores": 8
    }
}

In [ ]:
vm_family = "Dv2"
node_count = 2
requested_cores = node_count * vm_dict[vm_family]["cores"]

In [ ]:
results = subprocess.run([
    "az", "vm", "list-usage", 
    "--location", get_key(env_path, "aks_location"), 
    "--query", "[?contains(localName, '%s')].{max:limit, current:currentValue}" % (vm_family)
], stdout=subprocess.PIPE)
quota = json.loads(''.join(results.stdout.decode('utf-8')))
diff = int(quota[0]['max']) - int(quota[0]['current'])

In [ ]:
if diff <= requested_cores:
    print("Not enough cores of NCSv3 in region, asking for {} but have {}".format(requested_cores, diff))
    raise Exception("Core Limit", "Note enough cores to satisfy request")
print("There are enough cores, you may continue...")

In [ ]:
# Use a configuration of 2 VMs
prov_config = AksCompute.provisioning_configuration(
    agent_count=2, vm_size="Standard_D4_v2", location=aks_location
)

# Create the cluster
aks_target = ComputeTarget.create(
    workspace=ws, name=aks_name, provisioning_configuration=prov_config
)

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Let's check that the cluster is created successfully.

In [ ]:
aks_status = aks_target.get_status()

In [ ]:
assert aks_status == 'Succeeded', 'AKS failed to create'

## Deploy web service to AKS

Next, we deploy the web service. We deploy two pods with 1 CPU core each.

In [ ]:
#Set the web service configuration 
aks_config = AksWebservice.deploy_configuration(num_replicas=2, cpu_cores=1)

In [ ]:
%%time
aks_service = Webservice.deploy_from_image(
    workspace=ws,
    name=aks_service_name,
    image=image,
    deployment_config=aks_config,
    deployment_target=aks_target,
)
aks_service.wait_for_deployment(show_output=True)
print(aks_service.state)

You can check the logs of the web service with the below.

In [ ]:
aks_service.get_logs()

## Test the web service
We now test the web sevice.

In [ ]:
%%time
prediction = aks_service.run(input_data = jsontext)
print(prediction)

Let's try a few more duplicate questions and display their top 3 original matches. Let's first get the scoring URL and and API key for the web service.

In [ ]:
scoring_url = aks_service.scoring_uri
api_key = aks_service.get_keys()[0]

In [ ]:
headers = {'content-type': 'application/json', 'Authorization':('Bearer '+ api_key)}
r = requests.post(scoring_url, data=jsontext, headers=headers) # Run the request twice since the first time takes a 
%time r = requests.post(scoring_url, data=jsontext, headers=headers) # little longer due to the loading of the model
print(r)
r.json()

In [ ]:
dupes_to_score = dupes_test.iloc[:5,4]

In [ ]:
results = [
    requests.post(scoring_url, data=text_to_json(text), headers=headers)
    for text in dupes_to_score
]

Let's print top 3 matches for each duplicate question.

In [ ]:
[eval(results[i].json())[0:3] for i in range(0, len(results))]

Next let's quickly check what the request response performance is for the deployed model on AKS cluster.

In [ ]:
text_data = list(map(text_to_json, dupes_to_score))  # Retrieve the text data

In [ ]:
timer_results = list()
for text in text_data:
    res=%timeit -r 1 -o -q requests.post(scoring_url, data=text, headers=headers)
    timer_results.append(res.best)

In [ ]:
timer_results

In [ ]:
print("Average time taken: {0:4.2f} ms".format(10 ** 3 * np.mean(timer_results)))

## Scaling

In this part, we scale the number of pods to make sure we fully utilize the AKS cluster. To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [ ]:
!sudo az aks install-cli

Next, we will get the credentials to connect to the cluster.

In [ ]:
os.makedirs(os.path.join(os.path.expanduser('~'),'.kube'), exist_ok=True) 

In [ ]:
config_path = os.path.join(os.path.expanduser('~'),'.kube/config')

In [ ]:
with open(config_path, 'a') as f:
    f.write(aks_target.get_credentials()['userKubeConfig'])

Let's check the nodes and pods of the cluster.

In [ ]:
!kubectl get nodes

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
!kubectl get events

We can now scale up the number of pods.

In [ ]:
!kubectl scale --current-replicas=2 --replicas=10 {"deployment/" + aks_service_name}

In [ ]:
!kubectl get pods --all-namespaces

In [ ]:
!kubectl get deployment

Next, we will test the [throughput of the web service](05_Speed_Test_WebApp.ipynb).